# Passive suicidality in a repressive U.S. political context: Aim I


_WIP - NOT FOR DISTRIBUTION_

_Imports, re-indexes by date, cleans, reduces, restricts by timeframe; permits regex pattern-matched purposive (Wave 1) and random (Wave 2) sampling and named entity redaction of PushShift .gzip Reddit archives for .xlsx annotation. Computes Cohen's Kappa post-annotation. Permits LLM-assisted per-tag triangulation of annotation discrepancies._

> aim_i_annotate_triangulate_iaa.ipynb<br>
> Simone J. Skeen (09-13-2024)

1. [Prepare](xx)
2. [Pre-annotation](xx)
3. [Wave 1 (purposive)](xx)
4. [Wave 2 (random)](xx)
5. [Post-annotation](xx)
6. [Human-LLM triangulation](xx)

### Prepare

In [ ]:
    ### SJS 8/17: recreate using "New Colab" in Drive, reorder logically for GitHub + clean up Fx

In [ ]:
#%%capture

%pip install irrCAC

%pip install openai
%pip install --upgrade openai

%pip install --upgrade pydantic

    ### SJS 8/13: dependency error - fix this

#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.0.0/en_core_web_lg-3.0.0.tar.gz

In [ ]:
#import en_core_web_lg
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
#import openai
import os
import pandas as pd
import random
import re
import spacy
import time
import warnings
import webbrowser

from collections import Counter
from google.colab import drive
from irrCAC.raw import CAC
#from openai import OpenAI
from sklearn.metrics import cohen_kappa_score
from sklearn.utils import shuffle

#spacy.cli.download('en_core_web_lg')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option(
              'display.max_columns',
              None,
              )
pd.set_option(
              'display.max_rows',
              None,
              )

warnings.simplefilter(
                      action = 'ignore',
                      category = FutureWarning,
                      )

#!python -m prodigy stats

In [ ]:
drive.mount(
            '/content/drive',
            #force_remount = True,
            )

### Pre-annotation
***

**_Import_**

In [ ]:
wd = '/content/drive/MyDrive/Colab/dissertation/d_posts' ### Colab - gdrive

#wd = 'C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/03_prospectus/d_posts' ### Jupyter - local

ds = []

# loop over .json.gz

for filename in os.listdir(wd):
    if filename.endswith('.json.gz'):
        filepath = os.path.join(
                                wd,
                                filename,
                                )
        with gzip.open(
                       filepath,
                       'rt', ### 'open for reading', 'text mode'
                       encoding = 'utf-8',
                       ) as i:
            data = [json.loads(line) for line in i]
            d_gz = pd.DataFrame(data)
            ds.append(d_gz)

# concatenate

d = pd.concat(
              ds,
              ignore_index = True,
              )

# harmonize

d = d.dropna(
             axis = 1,
             how = 'any',
             )

# de-duplicate

d = d.drop_duplicates(
                      subset = 'id',
                      )

# re-index

d['date'] = pd.to_datetime(
                           d.created_utc,
                           unit = 's',
                           )

d.set_index(
            'date',
            drop = False,
            inplace = True,
            )

# inspect

d.shape
d.dtypes
d.sample(3)

**_Format_**

In [ ]:
def prep(d):
    '''Prepares d for annotation'''
    d = d[[
           'author',
           'created_utc',
           'date',
           'id',
           'num_comments',
           'selftext',
           'subreddit',
           'title',
           ]].copy()

    d.rename(
             columns = {
                        'author': 'p_au',
                        'created_utc': 'p_utc',
                        'date': 'p_date',
                        'id': 'p_id',
                        'num_comments': 'n_cmnt',
                        'selftext': 'text',
                        'subreddit': 'sbrt',
                        'title': 'p_titl',
                        }, inplace = True,
            )

    d = d.assign(
                 asp = ' ',      ### STS
                 asp_rtnl = ' ',
                 dep = ' ',
                 dep_rtnl = ' ',
                 val = ' ',
                 val_rtnl = ' ',
                 prg = ' ',      ### explicit
                 tgd = ' ',
                 age = ' ',      ### hybrid
                 race = ' ',     ### implicit
                 dbty = ' ',
                 insb = ' ',     ### insubstantial
                 )

    d = d[~d['text'].isin([
                           '[deleted]',
                           '[removed]',
                           ])]

    return d


In [ ]:

d = prep(d)

# restrict timeframe

d = d.loc[(d['p_date'] >= '2022-01-01') & (d['p_date'] <= '2022-12-31')]

# verify

d.shape
sbrt = d['sbrt'].unique()
print(sbrt)
d.head(1)
d.tail(1)

# plot

monthly_counts = d.resample('M').sbrt.value_counts().unstack().fillna(0)

fig, ax = plt.subplots(figsize=(10, 6))

monthly_counts.plot(
                    kind = 'line',
                    ax = ax,
                    )

box = ax.get_position()

ax.set_position(
                [
                 box.x0,
                 box.y0,
                 box.width * 0.8,
                 box.height,
                 ]
                  )

ax.legend(
          loc = 'center left',
          bbox_to_anchor=(1, 0.5),
          )

plt.show()

**_Parse by subreddit_**

In [ ]:
def parse(d, col):
    '''Parses d by subreddit'''
    uniq_val = d[col].unique()
    sub_d = {}
    for val in uniq_val:
        sub_d[f'd_{val}'] = d[d[col] == val].copy()

    return sub_d

sub_d = parse(
              d,
              'sbrt',
              )

**_Subset A: strains (proxy)_**

In [ ]:
#d_ax = sub_d['d_Anxiety'] ### deprecated
d_dp = sub_d['d_depression']
#d_mh = sub_d['d_mentalhealth'] ### deprecated
d_sw = sub_d['d_SuicideWatch']

#print('r/Anxiety')
#d_ax.shape
print("\nr/depression")
d_dp.shape
#print("\nr/mentalhealth")
#d_mh.shape
print("\nr/SuicideWatch")
d_sw.shape

**_Subset B: explicit BAR targets (proxy)_**

In [ ]:
d_gs = sub_d['d_TheGirlSurvivalGuide']
d_tr = sub_d['d_trans']
d_tx = sub_d['d_TwoXChromosomes']

print("r/TheGirlSurvivalGuide")
d_gs.shape
print("\nr/Trans")
d_tr.shape
print("\nr/TwoXChromosomes")
d_tx.shape

**_ner_redact_post_texts_**

In [ ]:

nlp = spacy.load('en_core_web_lg')

def ner_redact_post_texts(p_text):
    ne = list(
              [
               'PERSON',   ### people, including fictional
               'NORP',     ### nationalities or religious or political groups
               'FAC',      ### buildings, airports, highways, bridges, etc.
               'ORG',      ### companies, agencies, institutions, etc.
               #'GPE',     ### countries, cities, states
               'LOC',      ### non-GPE locations, mountain ranges, bodies of water
               'PRODUCT',  ### objects, vehicles, foods, etc. (not services)
               'EVENT',    ### named hurricanes, battles, wars, sports events, etc.
               ]
                )

    doc = nlp(p_text)
    ne_to_remove = []
    final_string = str(p_text)
    for sent in doc.ents:
        if sent.label_ in ne:
            ne_to_remove.append(str(sent.text))
    for n in range(len(ne_to_remove)):
        final_string = final_string.replace(
                                            ne_to_remove[n],
                                            '<|PII|>',
                                            )
    return final_string

#d['text'] = d['text'].astype(str).apply(lambda i: ner_redact_post_texts(i))

### Wave 1: purposive

#### Cycle 0: pilot (_n_ = 100)

**_r/SuicideWatch_**

In [ ]:
'.gend\S*|pregnan\S*' ### a priori/canonical
'trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe' ### inductively derived

rg = re.compile('.gend\S*|pregnan\S*|trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe', re.I)

d = d.loc[d['text'].str.contains(
                                 rg,
                                 regex = True,
                                 )]

d['text'] = d['text'].astype(str).apply(lambda i: redact(i))
d.shape

# export: 'd_cycle*.xlsx'

os.chdir('C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/03_prospectus/annotation/d_raw')
#%pwd

d.to_excel('d_cycle0.xlsx')

#### Cycle 1 (_n_ = 100)

**Subset A**<br>
r/Anxiety, r/Depression, r/MentalHealth, r/SuicideWatch

**Subset B**<br>
r/Trans

In [ ]:
d_a = pd.concat([
                 d_ax,
                 d_dp,
                 d_mh,
                 d_sw,
                 ])

d_a.reset_index(
                drop = True,
                inplace = True,
                )

d_b = d_tr.copy()

In [ ]:
# subset A: r/anxiety, r/depression, r/mentalhealth, r/SuicideWatch

'.gend\S*|pregnan\S*' ### a priori/canonical
'trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe' ### inductively derived

rg_a = re.compile('.gend\S*|pregnan\S*|trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe', re.I)

d_a = d_a.loc[d_a['text'].str.contains(
                                       rg_a,
                                       regex = True,
                                       )]

d_a.shape

# subset B: r/trans

'.criminal\S*|restrict\S*|.law|.legal\S*' ### a priori/canonical

rg_b = re.compile('.criminal\S*|restrict\S*|.law|.legal\S*', re.I)

d_b = d_b.loc[d_b['text'].str.contains(
                                       rg_b,
                                       regex = True,
                                       )]

d_b.shape

In [ ]:
d = pd.concat([
               d_a, # n = 9740
               d_b, # n = 1505
               ])

d.shape # N = 11245

**Export: 'd_cycle*.xlsx'**

In [ ]:
d = d.sample(n = 100)

d['text'] = d['text'].astype(str).apply(lambda i: redact(i))
d.shape

# export: 'd_cycle*.xlsx'

os.chdir('C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/03_prospectus/annotation/d_raw')
#%pwd

d.to_excel('d_cycle1.xlsx')

####Cycle 2 (_n_ = 100)

**Subset A**<br>
r/Depression, r/SuicideWatch<br>

**Subset B**<br>
r/Trans

In [ ]:
d_a = pd.concat([
                 d_dp,
                 d_sw,
                 ])

        ### SJS 5/22: dropping r/Anxiety, r/mentalhealth

d_a.reset_index(
                drop = True,
                inplace = True,
                )

d_b = d_tr.copy()

In [ ]:
# subset A

'.gend\S*|pregnan\S*' ### a priori/canonical
'trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe' ### inductively derived

rg_a = re.compile('.gend\S*|pregnan\S*|trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe', re.I)

d_a = d_a.loc[d_a['text'].str.contains(
                                       rg_a,
                                       regex = True,
                                       )]

d_a.shape

# subset B

'.criminal\S*|restrict\S*|illegal\S*|outlaw\S*|suicid\S*' ### a priori/canonical

rg_b = re.compile('.criminal\S*|restrict\S*|illegal\S*|outlaw\S*|suicid\S*', re.I)

d_b = d_b.loc[d_b['text'].str.contains(
                                       rg_b,
                                       regex = True,
                                       )]

d_b.shape

In [ ]:
d = pd.concat([
               d_a, # n = 5602
               d_b, # n = 729
               ])

d.shape # N = 11245

**Export: 'd_cycle*.xlsx'**

In [ ]:
d = d.sample(n = 100)

d['text'] = d['text'].astype(str).apply(lambda i: redact(i))

# export: 'd_cycle*.xlsx'

os.chdir('C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/03_prospectus/annotation/d_raw')
#%pwd

d.to_excel('d_cycle2.xlsx')

#### Cycle 3 (_n_ = 150)

**Subset A**<br>
r/Depression, r/SuicideWatch<br>

**Subset B**<br>
r/TheGirlsSurvivalGuide, r/Trans, r/TwoXChromosomes

In [ ]:
d_a = pd.concat([
                 d_dp,
                 d_sw,
                 ])

d_a.reset_index(
                drop = True,
                inplace = True,
                )

d_b = pd.concat([
                 d_gs,
                 d_tr,
                 d_tx,
                 ])

d_b.reset_index(
                drop = True,
                inplace = True,
                )

In [ ]:
# subset A

'.gend\S*|pregnan\S*' ### a priori/canonical
'trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe' ### inductively derived

rg_a = re.compile('.gend\S*|pregnan\S*|trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe', re.I)

d_a = d_a.loc[d_a['text'].str.contains(
                                       rg_a,
                                       regex = True,
                                       )]

d_a.shape

# subset B

'.criminal\S*|restrict\S*|illegal\S*|outlaw\S*|suicid\S*' ### a priori/canonical

rg_b = re.compile('.criminal\S*|restrict\S*|illegal\S*|outlaw\S*|suicid\S*', re.I)

d_b = d_b.loc[d_b['text'].str.contains(
                                       rg_b,
                                       regex = True,
                                       )]

d_b.shape

In [ ]:
d = pd.concat([
               d_a, # n = 5602
               d_b, # n = 1971
               ])

d.shape # N = 7573

**Export: 'd_cycle*.xlsx'**

In [ ]:
d = d.sample(n = 150)

d['text'] = d['text'].astype(str).apply(lambda i: redact(i))

#os.chdir('C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/03_prospectus/annotation/d_raw')
#%pwd

%cd gdrive/My Drive/Colab/prospectus/annotation/d_raw

d.to_excel('d_cycle3.xlsx')

In [ ]:
# random subset - Cycle 999

d_999 = d.sample(n = 1000)

d_999['text'] = d_999['text'].astype(str).apply(lambda i: redact(i))

d_999.to_excel('d_cycle999.xlsx')

#### Cycle 4 (_n_ = 150)

**Supplementing _prg_**

In [ ]:
# inspecting new _prg_ foregrounding subreddits...

    ### SJS 7/15: 2022 for now

%cd /content/gdrive/My Drive/Colab/dissertation/d_posts

d_aw = pd.read_json(
                    #'r_thegirlsurvivalguide_posts.jsonl', # d_gs.xlsx
                    #'r_confession_posts.jsonl', # d_co.xlsx
                    'r_askwomenadvice_posts.jsonl', # d_aw.xlsx
                    #'r_traumatoolbox_posts.jsonl', # d_tb.xlsx
                    #'r_birthcontrol_posts.jsonl', # d_bc.xlsx
                    #'r_WomensHealth_posts.jsonl', # d_wh.xlsx
                    lines = True,
                    )

    ### SJS 7/15: decision: adding r_askwomenadvice to Cycle 4

d_aw.shape
d_aw.head(3)
d_aw.tail(3)

In [ ]:
# harmonize

d_aw = d_aw.dropna(
                   axis = 1,
                   how = 'any',
                   )

# de-duplicate

d_aw = d_aw.drop_duplicates(
                            subset = 'id',
                            )

# re-index

d_aw['date'] = pd.to_datetime(
                              d_aw.created_utc,
                              unit = 's',
                              )

d_aw.set_index(
               'date',
               drop = False,
               inplace = True,
               )

#d.shape
#d.head(3)

d_aw = prep(d_aw)

#d.shape
#d.head(3)

d_aw['text'] = d_aw['text'].astype(str).apply(lambda i: redact(i))
#d_gs['text'] = d_gs['text'].astype(str).apply(lambda i: redact(i))

d_aw.shape
d_aw.head(3)
d_aw.tail(3)

#d_gs.shape
#d_gs.head(3)


In [ ]:
d_dp.shape
d_dp.head(3)
d_sw.shape
d_sw.head(3)

d_gs.shape
d_gs.head(3)
d_tr.shape
d_tr.head(3)
d_tx.shape
d_tx.head(3)

**Subset A: strains**<br>
r/Depression, r/SuicideWatch<br>

**Subset B: explicit targeting**<br>
r/AskWomenAdvice, r/TheGirlsSurvivalGuide, r/Trans, r/TwoXChromosomes

In [ ]:
d_a = pd.concat([
                 d_dp,
                 d_sw,
                 ])

d_a.reset_index(
                drop = True,
                inplace = True,
                )

d_b = pd.concat([
                 d_tr,
                 d_tx,
                 ])

d_b.reset_index(
                drop = True,
                inplace = True,
                )

    ### SJS 7/16: giving an artifical boost to these two (n = 20)

d_c = pd.concat([
                 d_aw,
                 d_gs,
                 ])

d_c.reset_index(
                drop = True,
                inplace = True,
                )


In [ ]:
d_a = shuffle(d_a)
d_a.head(10)
#d_a.sample(10)

In [ ]:
d_b = shuffle(d_b)
d_b.head(10)
#d_b.sample(10)

In [ ]:
d_c = shuffle(d_c)
d_c.head(10)
#d_c.sample(10)

In [ ]:
# subset A

'.gend\S*|pregnan\S*' ### a priori/canonical
'trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe' ### inductively derived

rg_a = re.compile('.gend\S*|pregnan\S*|trans|non-?binary|dysphor\S*|hormone|abort\S*|dobbs|roe', re.I)

d_a = d_a.loc[d_a['text'].str.contains(
                                       rg_a,
                                       regex = True,
                                       )]

d_a.shape

# subset B+

'.criminal\S*|restrict\S*|illegal\S*|outlaw\S*|suicid\S*|dobbs|roe|pregnan\S*' ### a priori/canonical

rg_b = re.compile('.criminal\S*|restrict\S*|illegal\S*|outlaw\S*|suicid\S*|dobbs|roe|pregnan\S*', re.I)

d_b = d_b.loc[d_b['text'].str.contains(
                                       rg_b,
                                       regex = True,
                                       )]

d_b.shape

d_c = d_c.loc[d_c['text'].str.contains(
                                       rg_b,
                                       regex = True,
                                       )]

d_c.shape

In [ ]:
d = pd.concat([
               d_a, # n = 5602
               d_b, # n = 5478
               ])

d.shape # N = 11080

**Export: 'd_cycle*.xlsx'**

In [ ]:
d = d.sample(n = 130)

d['text'] = d['text'].astype(str).apply(lambda i: redact(i))


In [ ]:
# supplementing Cycle 4

d_suppl = d_c.sample(n = 20)

d_suppl['text'] = d_suppl['text'].astype(str).apply(lambda i: redact(i))

In [ ]:
# supplementing Cycle 999 (training data)

%cd /content/gdrive/My Drive/Colab/prospectus/annotation/d_raw

d_suppl = d_c.sample(n = 200)
#d_suppl['text'] = d_suppl['text'].astype(str).apply(lambda i: redact(i))
d_suppl.to_excel('d_cycle999_suppl.xlsx')

In [ ]:
d = pd.concat([
               d, # n = 130
               d_suppl, # n = 20
               ])

d.shape # N = xx

In [ ]:
d = shuffle(d)

In [ ]:
%cd /content/gdrive/My Drive/Colab/prospectus/annotation/d_raw

d.to_excel('d_cycle4.xlsx')

### Wave 2: random

In [ ]:
%cd /content/drive/My Drive/Colab/dissertation/d_posts

d_aw = pd.read_excel('d_aw.xlsx')

d_aw.shape
d_aw.head(3)
d_aw.tail(3)

In [ ]:
    ### SJS 8/15: same _prg_ oversampling strategy: 1000 random fr d_a and d_b, plus 200 from r/GSG and r/AWA

d_dp.shape
d_dp.head(3)
d_sw.shape
d_sw.head(3)

d_aw.shape
d_aw.head(3)
d_gs.shape
d_gs.head(3)
d_tr.shape
d_tr.head(3)
d_tx.shape
d_tx.head(3)

In [ ]:
d_a = pd.concat([
                 d_dp, ### r/depression
                 d_sw, ### r/SuicideWatch
                 ])

d_a = shuffle(d_a)

d_a.reset_index(
                drop = True,
                inplace = True,
                )

d_b = pd.concat([
                 d_tr, ### r/trans
                 d_tx, ### r/TwoXChromosomes
                 ])

d_b = shuffle(d_b)

d_b.reset_index(
                drop = True,
                inplace = True,
                )

d_c = pd.concat([
                 d_aw, ### r/AskWomenAdvice,
                 d_gs, ### r/TheGirlsSurvivalGuide
                 ])

d_c = shuffle(d_c)

d_c.reset_index(
                drop = True,
                inplace = True,
                )

d_a.shape
d_b.shape
d_c.shape

In [ ]:
d_a.head(3)
d_b.head(3)

#d_c = d_c.drop(
#               'date',
#               axis = 1,
#               )

d_c.head(3)

In [ ]:
d = pd.concat([
               d_a, # n = 178030
               d_b, # n = 110213
               ])

d.shape # N = 288243

In [ ]:
d = d.sample(n = 1000)

d['text'] = d['text'].astype(str).apply(lambda i: ner_redact_post_texts(i))

d.shape # N = 1000

In [ ]:
d_suppl = d_c.sample(n = 200)

d_suppl['text'] = d_suppl['text'].astype(str).apply(lambda i: ner_redact_post_texts(i))

d_suppl.shape # N = 200

In [ ]:
d = pd.concat([
               d, # n = 1000
               d_suppl, # n = 200
               ])

d.shape # N = 1200

In [ ]:
%cd /content/drive/My Drive/Colab/dissertation/annotation/d_raw

d.to_excel('d_cycle999_rnd.xlsx')

### Post-annotation
***

**_Import_**

In [ ]:

    ### SJS 7/23: old, but use for pre-negotiation kappa, then define GPT-4o Fx, then split and annotate d_dis etc

    ### SJS 7/23: convention: d_iaa_cycle* for pre-GPT; d_iaa_gpt_cycle* for post-GPT negotiated


%cd /content/drive/My Drive/Colab/bar_policy_suicidality/inputs/annotation

d_sd = pd.read_excel('d_cycle1_sd.xlsx')
#d_sd.dtypes

#d_sd = d_sd.replace(' ', 0)
d_sd.columns = [f'{col}_sd' for col in d_sd.columns]


d_ss = pd.read_excel('d_cycle1_ss.xlsx')
#d_ss.dtypes

#d_ss = d_ss.replace(' ', 0)
d_ss.columns = [f'{col}_ss' for col in d_ss.columns]

# inspect

#d_sd.head(3)
#d_ss.head(3)

# merge

d = pd.merge(
             d_sd,
             d_ss,
             left_index = True,
             right_index = True,
             )

targets = [
           'asp_sd', 'asp_ss',
           'dep_sd', 'dep_ss',
           'val_sd', 'val_ss',
           'prg_sd', 'prg_ss',
           'tgd_sd', 'tgd_ss',
           'age_sd', 'age_ss',
           'race_sd', 'race_ss',
           'dbty_sd', 'dbty_ss',
           'insb_sd', 'insb_ss',
            ]

#d = d[targets].copy()

d[targets] = d[targets].apply(
                              pd.to_numeric,
                              errors = 'coerce',
                              )

d = d.fillna(0)
d = d.replace(' ', 0)
#d = d.apply(pd.to_numeric, errors='coerce')


d = d[[
        #targets,
        'p_id_sd', 'p_id_ss', ### sense-check for bad merge
        'text_sd',
        'asp_sd', 'asp_ss',
        'asp_rtnl_sd', 'asp_rtnl_ss',
        'dep_sd', 'dep_ss',
        'dep_rtnl_sd', 'dep_rtnl_ss',
        'val_sd', 'val_ss',
        'val_rtnl_sd', 'val_rtnl_ss',
        'prg_sd', 'prg_ss',
        'tgd_sd', 'tgd_ss',
        'age_sd', 'age_ss',
        'race_sd', 'race_ss',
        'dbty_sd', 'dbty_ss',
        'insb_sd', 'insb_ss',
        ]].copy()

d.rename(
           columns = {
                      'p_id_sd': 'p_id',
                      'text_sd': 'text',
                      }, inplace = True,
          )

#d.to_excel('iaa_inspect.xlsx')

# drop insb = 1 (either annotator)

#d = d[(d['insb_sd'] != 1) & (d['insb_ss'] != 1)]

# target = 0 for insb = 1 (either annotator)

# condition to check where either 'insb_sd' or 'insb_ss' is 1

condition = (d['insb_sd'] == 1) | (d['insb_ss'] == 1)

# update 'asp', 'dep', and 'val' to 0 where the condition is True

d.loc[condition, ['asp', 'val', 'dep']] = 0


'n valid'
d.shape

# convert to numpy array

#for target in targets:
#    target = d[target].to_numpy()

# kappa Fx

def calculate_kappa(d, col_sd, col_ss):
    return cohen_kappa_score(d[col_sd], d[col_ss])

# initialize SD-SJS col pair list

column_pairs = [
                ('asp_sd', 'asp_ss'),
                ('dep_sd', 'dep_ss'),
                ('val_sd', 'val_ss'),
                #('prg_sd', 'prg_ss'),
                #('tgd_sd', 'tgd_ss'),
                #('age_sd', 'age_ss'),
                #('race_sd', 'race_ss'),
                #('dbty_sd', 'dbty_ss'),
                ]

# initialize dict

kappa_results = {}

# kappa loop

for col_sd, col_ss in column_pairs:
    kappa = calculate_kappa(d, col_sd, col_ss)
    kappa_results[f'{col_sd} and {col_ss}'] = kappa

for pair, kappa in kappa_results.items():
    print(f"Cohen's Kappa for {pair}: {kappa:.2f}")


# dummy disagreement Fx

def d_disagree(row):
    return 1 if row[0] != row[1] else 0

sts_pairs = [('asp_sd', 'asp_ss', 'asp_dis'),
             ('dep_sd', 'dep_ss', 'dep_dis'),
             ('val_sd', 'val_ss', 'val_dis')]

for col1, col2, dis_col in sts_pairs:
    d[dis_col] = d[[col1, col2]].apply(d_disagree, axis = 1)

#d.to_excel('iaa_cycle3.xlsx')

In [ ]:
%cd /content/drive/My Drive/Colab/bar_policy_suicidality/inputs/annotation

d_sd = pd.read_excel('d_cycle4_sd.xlsx')
# d_sd.dtypes

# d_sd = d_sd.replace(' ', 0)
d_sd.columns = [f'{col}_sd' for col in d_sd.columns]

d_ss = pd.read_excel('d_cycle4_ss.xlsx')
# d_ss.dtypes

# d_ss = d_ss.replace(' ', 0)
d_ss.columns = [f'{col}_ss' for col in d_ss.columns]

# inspect

# d_sd.head(3)
# d_ss.head(3)

# merge

d = pd.merge(
    d_sd,
    d_ss,
    left_index=True,
    right_index=True,
)

targets = [
    'asp_sd', 'asp_ss',
    'dep_sd', 'dep_ss',
    'val_sd', 'val_ss',
    'prg_sd', 'prg_ss',
    'tgd_sd', 'tgd_ss',
    'age_sd', 'age_ss',
    'race_sd', 'race_ss',
    'dbty_sd', 'dbty_ss',
    'insb_sd', 'insb_ss',
]

# d = d[targets].copy()

d[targets] = d[targets].apply(
    pd.to_numeric,
    errors='coerce',
)

d = d.fillna(0)
d = d.replace(' ', 0)
# d = d.apply(pd.to_numeric, errors='coerce')

d = d[[
    # targets,
    'p_id_sd', 'p_id_ss',  # sense-check for bad merge
    'text_sd',
    'asp_sd', 'asp_ss',
    'asp_rtnl_sd', 'asp_rtnl_ss',
    'dep_sd', 'dep_ss',
    'dep_rtnl_sd', 'dep_rtnl_ss',
    'val_sd', 'val_ss',
    'val_rtnl_sd', 'val_rtnl_ss',
    'prg_sd', 'prg_ss',
    'tgd_sd', 'tgd_ss',
    'age_sd', 'age_ss',
    'race_sd', 'race_ss',
    'dbty_sd', 'dbty_ss',
    'insb_sd', 'insb_ss',
]].copy()

d.rename(
    columns={
        'p_id_sd': 'p_id',
        'text_sd': 'text',
    }, inplace=True,
)

# d.to_excel('iaa_inspect.xlsx')

# drop insb = 1 (either annotator)

# d = d[(d['insb_sd'] != 1) & (d['insb_ss'] != 1)]

# target = 0 for insb = 1 (either annotator)

condition = (d['insb_sd'] == 1) | (d['insb_ss'] == 1)

# update 'asp', 'dep', and 'val' to 0 where insb = 1 (either annotator)

d.loc[condition, ['asp', 'val', 'dep']] = 0

'n valid'
d.shape

# to numpy

# for target in targets:
#     target = d[target].to_numpy()

# kappa Fx

def calculate_kappa(d, col_sd, col_ss):
    return cohen_kappa_score(d[col_sd], d[col_ss])

# initialize SD-SJS col pair list

column_pairs = [
    ('asp_sd', 'asp_ss'),
    ('dep_sd', 'dep_ss'),
    ('val_sd', 'val_ss'),
    # ('prg_sd', 'prg_ss'),
    # ('tgd_sd', 'tgd_ss'),
    # ('age_sd', 'age_ss'),
    # ('race_sd', 'race_ss'),
    # ('dbty_sd', 'dbty_ss'),
]

# initialize dict

kappa_results = {}

# kappa loop

for col_sd, col_ss in column_pairs:
    kappa = calculate_kappa(d, col_sd, col_ss)
    kappa_results[f'{col_sd} and {col_ss}'] = kappa

for pair, kappa in kappa_results.items():
    print(f"Cohen's Kappa for {pair}: {kappa:.2f}")


# dummy disagreement Fx

def d_disagree(row):
    return 1 if row[0] != row[1] else 0

sts_pairs = [('asp_sd', 'asp_ss', 'asp_dis'),
             #('dep_sd', 'dep_ss', 'dep_dis'),
             ('val_sd', 'val_ss', 'val_dis')]

for col1, col2, dis_col in sts_pairs:
    d[dis_col] = d[[col1, col2]].apply(d_disagree, axis=1)

# d.to_excel('iaa_cycle3.xlsx')

### Human-LLM triangulation

**_Formulate 'asp' prompt_**

In [ ]:
role = '''
You are tasked with applying qualitative codes to social media posts to categorize whether each post contains an expression of _aspiration strain_.
'''

definition = '''
Definition of aspiration strain: 'any description of ambition, futurity, idealized or speculative lifecourse trajectories, or personal,
professional, familial goals driving psychological strain and/or self-destructive cognitions.' Descriptions can be explicit or implicit.
'''

instruction = '''
Below I instruct on how to apply the codes.

Respond with 'asp_1' if the post contains an expression of aspiration strain, and '0' if it does not.

You must choose a 'asp_1' or a '0' response.

If your response is 'asp_1,' then begin a new paragraph with 'asp_rationale' and excerpt the sentences or phrases that are the _most expressive of
aspiration strain_. You are allowed to choose multiple sentences or phrases, divided by an '<SPL>' token.

If your response is 'asp_1,' then begin a new paragraph with 'strained aspirations:' and concisely name the strained aspiration that is driving the distressful
cognitions.

Then, whether you have selected a 'asp_1' or a '0,' begin a new paragraph with 'asp_explanation:' and provide a two sentence explanation for your response.
'''

clarification = '''
Here are additional clarifying points based in human expertise:
-	Regret over, or wishing to redo, a past decision does _not_ warrant a 'asp_1' response
-	Aspiration for physical impossibility (time travel, age reversion), does not warrant a 'asp_1' response
-	Sparse decontextualized expressions of loneliness (e.g. 'I am lonely') do not warrant a 'asp_1' response; recognition of need or yearning for
friendship, community, and/or intimacy must be explicit to warrant a 'asp_1' response
-	Sparse decontextualized desire for a different assigned sex at birth does not warrant a 'asp_1' response; desire for gender transition or
gender-expansive expression must be explicit.
- Perceived inability to die by suicide does not warrant a 'asp_1' response
'''

asp_prompt = f'{role}{definition}{instruction}{clarification}'
#print(prompt)

**_Formulate 'val' prompt_**

In [ ]:
role = '''
You are tasked with applying qualitative codes to social media posts to categorize whether each post contains an expression of _value strain_.
'''

definition = '''
Definition of aspiration strain: 'any description of conflicting ideologies, norms, mores, morals, ethics, principles, or ontologies, particularly
along traditionalist-authoritarian versus progressive-liberatory axes, driving intra-psychic strain and/or self-destructive cognitions..' Descriptions
can be explicit or implicit.
'''

instruction = '''
Below I instruct on how to apply the codes.

Respond with 'val_1' if the post contains an expression of value strain, and '0' if it does not.

You must choose a 'val_1' or a '0' response.

If your response is 'val_1,' then begin a new paragraph with 'val_rationale:' and excerpt the sentences or phrases that are the _most expressive of
value strain_. You are allowed to choose multiple sentences or phrases, divided by an '<SPL>' token.

If your response is 'val_1,' then begin a new paragraph with 'strained values:' and concisely name the strained value that is driving the distressful
cognitions.

Then, whether you have selected a 'val_1' or a '0,' begin a new paragraph with 'val_explanation:' and provide a two sentence explanation for your response.
'''

clarification = '''
Here are additional clarifying points based in human expertise:
-	a 'val_1' response refer to interpersonal ideological discord and/or internalized ideologically determined self-denigration
-	Anticipated value strain does warrant a 'val_1' response
-	Familial conflict alone is insufficient to warrant a 'val_1' response
-	Perceived failure to uphold self-imposed expectations is insufficient to warrant a 'val_1' response
-	Stress or tension over legality of decisions, necessities does warrant a 'val_1' response
-	ideological or normative aspect must be explicit
'''

val_prompt = f'{role}{definition}{instruction}{clarification}'
#print(prompt)

**_Define function_**

In [ ]:
api_key = '<api_key>' ### project = skeen_prospectus; name = annotate
client = OpenAI(api_key = api_key)

def annotate_post_per_tag(text):
    '''
    Applies an annotation decision, based on pre-specified prompt 'prompt', to a given text 'text'; provides rationale and explanation.
    '''
    try:
        response = client.chat.completions.create(
                                                  model = 'gpt-4o', ### https://platform.openai.com/docs/models
                                                  temperature = 0.2,
                                                  messages = [
                                                              {
                                                               'role': 'system',
                                                               'content': f'{asp_prompt}{val_prompt}' ### double up prompts?
                                                               },
                                                              {
                                                               'role': 'user',
                                                               'content': f'{text}'
                                                               },
                                                            ]
                                                  )
        result = ' '
        for choice in response.choices:
            result += choice.message.content
        print(f'{text}: {result}')
        return result
    except Exception as e:
        print(f'Exception: {e}')
        return 'error'

**annotate_post_per_tag**

In [ ]:
text = '''
All my life I wished I could be a champion snowboarder, but my injuries as a child ruined that dream.
Because I am medicated and rely on a walker to navigate only short distances, my athletic career can
never return. I feel hopeless and stunted and passed by, and like a burden to my family.

They maintain strict conservative ideals and are very traditional. I'm more forward-looking and accepting,
and for this too they make me feel like an outcast.
'''
annotate_post_per_tag(text)

In [ ]:
text = '''
I am a sick man.... I am a spiteful man. I am an unattractive man. I believe my liver is diseased.
However, I know nothing at all about my disease, and do not know for certain what ails me.
I don’t consult a doctor for it, and never have, though I have a respect for medicine and doctors.
Besides, I am extremely superstitious, sufficiently so to respect medicine, anyway (I am well-educated
enough not to be superstitious, but I am superstitious). No, I refuse to consult a doctor from spite.
That you probably will not understand. Well, I understand it, though. Of course, I can’t explain who
it is precisely that I am mortifying in this case by my spite: I am perfectly well aware that I cannot
“pay out” the doctors by not consulting them; I know better than anyone that by all this I am only
injuring myself and no one else. But still, if I don’t consult a doctor it is from spite. My liver is
bad, well—let it get worse!
'''
annotate_post_per_tag(text)

**annotate_dataframe_per_tag**

In [ ]:

#api_key = '<api_key>' ### project = skeen_prospectus; name = annotate
#client = OpenAI(api_key = api_key)

#def annotate_dataframe_per_tag(d):
#    '''
#    Applies annotate_post_per_tag to each row in dataframe 'd'.
#    '''
#    for index, row in d.iterrows():
#        result = annotate_post_per_tag(row['text'])
#        if result == 'error':
#            continue

    ### SJS 7/23: good snippet; keep for now

#        asp, rationale, strain, explanation = None, None, None, None

#        if '1' in result:
#            asp = 1
#            rationale = result.split('rationale:')[1].split('strain:')[0].strip() if 'rationale:' in result else None
#            strain = result.split('strain:')[1].split('explanation:')[0].strip() if 'strain:' in result else None
#        else:
#            asp = 0

#        explanation = result.split('explanation:')[1].strip() if 'explanation:' in result else None

#        d.at[index, 'asp_gpt'] = asp
#        d.at[index, 'asp_rtnl_gpt'] = rationale
#        d.at[index, 'asp_strn_gpt'] = strain
#        d.at[index, 'expl_gpt'] = explanation

        # delay between API calls
#        time.sleep(1)

#    return d


In [ ]:

api_key = '<api_key>'  # project = skeen_prospectus; name = annotate
client = OpenAI(api_key=api_key)

def annotate_post_per_tag(text):
    '''
    Applies an annotation decision, based on pre-specified prompt 'prompt', to a given text 'text'; provides rationale and explanation.
    '''
    try:
        response = client.chat.completions.create(
                                                  model = 'gpt-4o', ### https://platform.openai.com/docs/models
                                                  temperature = 0.2,
                                                  messages = [
                                                              {
                                                               'role': 'system',
                                                               'content': f'{asp_prompt}{val_prompt}' ### double up prompts?
                                                               },
                                                              {
                                                               'role': 'user',
                                                               'content': f'{text}'
                                                               },
                                                            ]
                                                  )
        result = ' '
        for choice in response.choices:
            result += choice.message.content
        print(f'{text}: {result}')
        return result
    except Exception as e:
        print(f'Exception: {e}')
        return 'error'
    pass

def annotate_dataframe_per_tag(d):
    '''
    Applies annotate_post_per_tag for aspiration and value strains to each row in dataframe 'd'.
    '''
    for index, row in d.iterrows():
        result = annotate_post_per_tag(row['text'])
        if result == 'error':
            continue

        # 'asp'

        asp, asp_rationale, asp_strain, asp_explanation = None, None, None, None

        if 'asp_1' in result:
            asp = 1
            asp_rationale = result.split('asp_rationale:')[1].split('strained aspirations:')[0].strip() if 'asp_rationale:' in result else None
            asp_strain = result.split('strained aspirations:')[1].split('asp_explanation:')[0].strip() if 'strained aspirations:' in result else None
            asp_explanation = result.split('asp_explanation:')[1].split('val_rationale:')[0].strip() if 'asp_explanation:' in result else None
        else:
            asp = 0

        d.at[index, 'asp_gpt'] = asp
        d.at[index, 'asp_rtnl_gpt'] = asp_rationale
        d.at[index, 'asp_strn_gpt'] = asp_strain
        d.at[index, 'asp_expl_gpt'] = asp_explanation

        # delay between API calls

        time.sleep(1)

        # 'val'

        val, val_rationale, val_strain, val_explanation = None, None, None, None

        if 'val_1' in result:
            val = 1
            val_rationale = result.split('val_rationale:')[1].split('strained values:')[0].strip() if 'val_rationale:' in result else None
            val_strain = result.split('strained values:')[1].split('val_explanation:')[0].strip() if 'strained values:' in result else None
            val_explanation = result.split('val_explanation:')[1].strip() if 'val_explanation:' in result else None
        else:
            val = 0

        d.at[index, 'val_gpt'] = val
        d.at[index, 'val_rtnl_gpt'] = val_rationale
        d.at[index, 'val_strn_gpt'] = val_strain
        d.at[index, 'val_expl_gpt'] = val_explanation

        # delay between API calls

        time.sleep(1)

    return d


In [ ]:
%cd /content/gdrive/My Drive/Colab/prospectus/annotation/gpt_triangulation_pilot

d_test = pd.read_excel('test.xlsx')
d_test.head(3)

In [ ]:
d_test = annotate_dataframe_per_tag(d_test)
d_test.head(3)

#### Cycle 3 LLM triangulation: pilot

In [ ]:
%cd /content/gdrive/My Drive/Colab/prospectus/annotation/llm_triangulation

d = pd.read_excel('iaa_cycle3.xlsx')

# focus on asp for pilot...

    ### SJS 7/15: reset index - later

d = d[[
       'p_id',
       'text',
       'asp_sd',
       'asp_ss',
       'asp_rtnl_sd',
       'asp_rtnl_ss',
       ]].copy()

d.shape
d.head(3)

In [ ]:
# triangulate Cycle 3

d = annotate_dataframe_per_tag(d)
#print(d)
d.head(3)


In [ ]:
d.to_excel('iaa-gpt_cycle3.xlsx')

#### Cycle 4 LLM triangulation: _asp_ and _val_

In [ ]:
%pwd

In [ ]:
%cd /content/drive/My Drive/Colab/bar_policy_suicidality/inputs/annotation

d_sd = pd.read_excel('d_cycle4_sd.xlsx')
#d_sd.dtypes

#d_sd = d_sd.replace(' ', 0)
d_sd.columns = [f'{col}_sd' for col in d_sd.columns]


d_ss = pd.read_excel('d_cycle4_ss.xlsx')
#d_ss.dtypes

#d_ss = d_ss.replace(' ', 0)
d_ss.columns = [f'{col}_ss' for col in d_ss.columns]


# inspect

#d_sd.head(3)
#d_ss.head(3)

# merge

d = pd.merge(
             d_sd,
             d_ss,
             left_index = True,
             right_index = True,
             )

targets = [
           'asp_sd', 'asp_ss',
           'dep_sd', 'dep_ss',
           'val_sd', 'val_ss',
           'prg_sd', 'prg_ss',
           'tgd_sd', 'tgd_ss',
           'age_sd', 'age_ss',
           'race_sd', 'race_ss',
           'dbty_sd', 'dbty_ss',
           'insb_sd', 'insb_ss',
            ]

#d = d[targets].copy()

d[targets] = d[targets].apply(
                              pd.to_numeric,
                              errors = 'coerce',
                              )

d = d.fillna(0)
d = d.replace(' ', 0)
#d = d.apply(pd.to_numeric, errors='coerce')


d = d[[
        #targets,
        'p_id_sd', 'p_id_ss', ### sense-check for bad merge
        'text_sd',
        'asp_sd', 'asp_ss',
        'asp_rtnl_sd', 'asp_rtnl_ss',
        'asp_strn_sd', 'asp_strn_ss', ### added _strn 'name the aspiration'
        'dep_sd', 'dep_ss',
        'dep_rtnl_sd', 'dep_rtnl_ss',
        'val_sd', 'val_ss',
        'val_rtnl_sd', 'val_rtnl_ss',
        'val_strn_sd', 'val_strn_ss', ### added _strn 'name the value'
        'prg_sd', 'prg_ss',
        'tgd_sd', 'tgd_ss',
        'age_sd', 'age_ss',
        'race_sd', 'race_ss',
        'dbty_sd', 'dbty_ss',
        'insb_sd', 'insb_ss',
        ]].copy()

d.rename(
           columns = {
                      'p_id_sd': 'p_id',
                      'text_sd': 'text',
                      }, inplace = True,
          )

#d.to_excel('iaa_inspect.xlsx')

# drop insb = 1 (either annotator)

#d = d[(d['insb_sd'] != 1) & (d['insb_ss'] != 1)]

# target = 0 for insb = 1 (either annotator)

# check: 'insb_sd' or 'insb_ss' is 1

condition = (d['insb_sd'] == 1) | (d['insb_ss'] == 1)

# 'asp', 'dep', and 'val' = 0 where 'insb' = True

d.loc[condition, ['asp', 'val', 'dep']] = 0


'n valid'
d.shape

# to numpy

#for target in targets:
#    target = d[target].to_numpy()

# kappa Fx

def calculate_kappa(d, col_sd, col_ss):
    return cohen_kappa_score(d[col_sd], d[col_ss])

column_pairs = [
                ('asp_sd', 'asp_ss'),
                ('dep_sd', 'dep_ss'),
                ('val_sd', 'val_ss'),
                #('prg_sd', 'prg_ss'),
                #('tgd_sd', 'tgd_ss'),
                #('age_sd', 'age_ss'),
                #('race_sd', 'race_ss'),
                #('dbty_sd', 'dbty_ss'),
                ]

kappa_results = {}

for col_sd, col_ss in column_pairs:
    kappa = calculate_kappa(d, col_sd, col_ss)
    kappa_results[f'{col_sd} and {col_ss}'] = kappa

for pair, kappa in kappa_results.items():
    print(f"Cohen's Kappa for {pair}: {kappa:.2f}")


# disagreement Fx

def d_disagree(row):
    return 1 if row[0] != row[1] else 0

sts_pairs = [('asp_sd', 'asp_ss', 'asp_dis'),
             ('dep_sd', 'dep_ss', 'dep_dis'),
             ('val_sd', 'val_ss', 'val_dis')]

for col1, col2, dis_col in sts_pairs:
    d[dis_col] = d[[col1, col2]].apply(d_disagree, axis = 1)

#d.to_excel('iaa_cycle3.xlsx')
#d.dtypes
#d.head(3)

In [ ]:
#d.shape
#d.dtypes
#d.head(3)

In [ ]:

# df for SJS-SD disagreements

d_dis = d[(d['asp_dis'] == 1) | (d['val_dis'] == 1)]
d_dis.shape

# drop disagreements fr orig df

d_agr = d[~((d['asp_dis'] == 1) | (d['val_dis'] == 1))]
d_agr.shape

# GPT-4o - asp

d_dis = annotate_dataframe_per_tag(d_dis)

# append d_dis to d_agr

d_iaa_gpt_cycle4 = pd.concat(
                             [
                              d_dis,
                              d_agr,
                              ], ignore_index = True,
                             )

# prioritize GPT-4o-annotated rows for human inspection

d_iaa_gpt_cycle4 = d_iaa_gpt_cycle4.sort_values(
                                                by = [
                                                      'asp_dis',
                                                      'val_dis',
                                                      ], ascending = False
                                                ).reset_index(drop = True)

d_iaa_gpt_cycle4.head(3)

In [ ]:
# sort cols for human readability + add 'notes'

d_iaa_gpt_cycle4['notes_for_asp'] = ' '
d_iaa_gpt_cycle4['notes_for_val'] = ' '

d_iaa_gpt_cycle4 = d_iaa_gpt_cycle4[[
                                     'text',
                                     'asp_sd',
                                     'asp_ss',
                                     'asp_gpt',
                                     'asp_rtnl_sd',
                                     'asp_rtnl_ss',
                                     'asp_rtnl_gpt',
                                     'asp_strn_sd',
                                     'asp_strn_ss',
                                     'asp_strn_gpt',
                                     'asp_expl_gpt',
                                     'asp_dis',
                                     'notes_for_asp',

                                     'text',
                                     'val_sd',
                                     'val_ss',
                                     'val_gpt',
                                     'val_rtnl_sd',
                                     'val_rtnl_ss',
                                     'val_rtnl_gpt',
                                     'val_strn_sd',
                                     'val_strn_ss',
                                     'val_strn_gpt',
                                     'val_expl_gpt',
                                     'val_dis',
                                     'notes_for_val',
                                     ]].copy()

d_iaa_gpt_cycle4.shape
d_iaa_gpt_cycle4.head(3)

In [ ]:
d_iaa_gpt_cycle4.to_excel('d_iaa_gpt_cycle4.xlsx')

#### Cycle 4: Post-LLM triangulation: _asp_ and _val_

In [ ]:
%cd /content/drive/My Drive/Colab/dissertation/annotation

d = pd.read_excel('d_iaa_gpt_cycle4_sd-ss.xlsx')

def calculate_kappa(d, col_sd, col_ss):
    return cohen_kappa_score(d[col_sd], d[col_ss])

# cols: SJS-SD

column_pairs = [
                ('asp_sd', 'asp_ss'),
                ('val_sd', 'val_ss'),
                ]

kappa_results = {}

for col_sd, col_ss in column_pairs:
    kappa = calculate_kappa(d, col_sd, col_ss)
    kappa_results[f'{col_sd} and {col_ss}'] = kappa

for pair, kappa in kappa_results.items():
    print(f"Cohen's Kappa for {pair}: {kappa:.2f}")


### Viz

In [ ]:
%cd /content/drive/My Drive/Colab/bar_policy_suicidality/outputs/tables

d_v = pd.read_excel('d_cycle_kappas.xlsx')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%cd .../figures

sns.set(style = 'whitegrid')

# custom x-axis labels

custom_labels = [
                 'cycle 1',
                 'cycle 2',
                 'cycle 3',
                 'pre-GTP: cycle 4',
                 'post-GPT: cycle 4',
                 ]

# plot

plt.figure(figsize = (
                      8,
                      6,
                      )
          )

plt.plot(
         d_v['cycle'],
         d_v['asp_cohens_k'],
         label = "asp",
         marker = 's',
         alpha = 0.6,
         color = 'hotpink',
         )

plt.plot(
         d_v['cycle'],
         d_v['dep_cohens_k'],
         label = "dep",
         marker = 's',
         alpha = 0.6,
         color = 'tomato',
         )

plt.plot(
         d_v['cycle'],
         d_v['val_cohens_k'],
         label = "val",
         marker = 's',
         alpha = 0.6,
         color = 'mediumorchid',
         )

# labels, title

#plt.xlabel('Cycle')
plt.ylabel("Cohen's $\kappa$")
#plt.title("Cohen's $\kappa$ Value per Cycle for ASP, DEP, and VAL")

# custom x-axis labels, 45-degree angle

plt.xticks(
           ticks = d_v['cycle'],
           labels=custom_labels,
           rotation = 45,
           )



# horizontal gridlines

plt.grid(axis='x')

# set line at 0.7 threshold

plt.axhline(
            y = 0.7,
            color = 'red',
            linewidth = 0.6,
            linestyle = '--',
            )

# x-axis at 0

plt.ylim(
         0,
         None,
         )

# legend

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=3, frameon=False)

# x-axis ticks

plt.gca().tick_params(axis='x', which='both', direction='in', length=5)  # Adds ticks to the x-axis labels

# optimized markers

#x0 = [2, 5, 5]
#y0 = [0.72, 0.96, 0.97]
#plt.plot(x0, y0, "s", markersize = 7, color = 'red')

#for a,b in zip(x0, y0):
#    plt.text(a, b, str(b),fontsize=9, ha='right',va='top')

sns.despine(
            left = True,
            )

# tight layout

plt.tight_layout()

# save

plt.savefig('cycle_kappa_line.png')

# display

plt.show()


> End of bar_sample_annotate_iaa.ipynb